# Agent Simulation: Persona-Based Behavioral Testing

**Objective**: Instantiate Claude-powered agents from persona profiles and test their responses to product scenarios.

## Workflow
1. Setup — Import agents module, load personas
2. Define Scenarios — Product offers designed to probe persona differences
3. Run Simulations — Execute scenarios across all 7 personas
4. Analyze Responses — Compare decisions and reasoning
5. Validate Differentiation — Verify personas respond consistently with their profiles

## 1. Environment Setup

In [ ]:
import sys
from pathlib import Path

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

from agents import PersonaSimulator, PersonaAgent

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
pd.set_option('display.max_colwidth', 200)

# Output directory
OUTPUT_DIR = Path('./outputs/05_agent_simulation')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR.resolve()}")

## 2. Initialize Simulator

Set `mock_mode=True` to test without API access. Switch to `False` when you have an API key configured.

In [ ]:
# Initialize simulator
# Set mock_mode=False when ANTHROPIC_API_KEY is configured
simulator = PersonaSimulator(
    personas_path='../data/processed/personas.json',
    mock_mode=True  # Change to False for real API calls
)

# Load personas and initialize agents
simulator.load_personas()
simulator.initialize_agents()

# Display persona summary
print("Loaded personas:")
simulator.get_persona_summary()

In [ ]:
# Quick test: Single agent response
test_agent = simulator.agents[0]  # Mainstream Shopper
print(f"Testing: {test_agent.persona_name}")
print("-" * 50)
response = test_agent.respond("Would you buy a R$150 headphone with free shipping?")
print(response)

## 3. Define Test Scenarios

Each scenario is designed to probe specific persona differences:

| Scenario | Tests | Expected Differentiation |
|----------|-------|--------------------------|
| Financing offer | Installment preference | High-Value Financing vs Cash Customer |
| Flash sale | Urgency, price sensitivity | Budget vs Premium personas |
| Bundle deal | Basket behavior | Bulk Buyer vs single-item personas |
| New category | Exploration tendency | Loyal Explorer vs Focused personas |
| Mixed reviews | Risk tolerance | Critical Shopper vs Satisfied personas |
| Weekend promotion | Shopping timing | Weekend Buyer vs Weekday shoppers |

In [ ]:
SCENARIOS = [
    {
        "name": "financing_offer",
        "text": """You're looking at a smartphone priced at R$1,200. 
The seller offers 10x installments of R$120 with no interest on credit card.
Alternatively, you can pay R$1,080 upfront via boleto (10% discount).
The phone has good reviews (4.2 stars) and ships in 3 days.
Would you buy this phone? Which payment option would you choose?"""
    },
    {
        "name": "flash_sale",
        "text": """FLASH SALE: A kitchen blender you've been eyeing is now 40% off!
Original price: R$250 → Sale price: R$150
The catch: Sale ends in 2 hours. You weren't planning to buy today.
Reviews are positive (4.5 stars), and you've needed a blender for a while.
Do you buy it now, or wait and risk missing the deal?"""
    },
    {
        "name": "bundle_deal",
        "text": """You need to buy replacement ink cartridges for your printer.
Single cartridge: R$45 each
Bundle of 3: R$115 (save 15%)
You definitely need 1 now, might need the others in 3-6 months.
Would you buy the single cartridge or the bundle?"""
    },
    {
        "name": "new_category",
        "text": """You've never bought fitness equipment online before, but you see an ad:
Adjustable dumbbells set for R$280 with 4.3 star reviews.
You've been thinking about working out at home.
This marketplace has always worked well for your electronics purchases.
Would you try buying fitness equipment from this platform?"""
    },
    {
        "name": "mixed_reviews",
        "text": """You found the exact wireless earbuds you want at R$180.
However, the reviews are mixed: 3.5 stars overall.
Positive reviews praise sound quality. Negative reviews mention Bluetooth connectivity issues.
The seller has a 30-day return policy.
Would you take the risk on these earbuds?"""
    },
    {
        "name": "weekend_promo",
        "text": """It's Tuesday, and you see a promotion:
'WEEKEND SPECIAL: 20% off all home decor items, Saturday-Sunday only!'
There's a decorative lamp you like for R$120 (would be R$96 with discount).
You could buy now at full price and get it by Thursday,
or wait for the weekend, buy at discount, and receive it next week.
What would you do?"""
    }
]

print(f"Defined {len(SCENARIOS)} test scenarios:")
for s in SCENARIOS:
    print(f"  - {s['name']}")

## 4. Run Simulations

In [ ]:
# Run all scenarios across all personas
print("Running simulations...")
print(f"  {len(SCENARIOS)} scenarios × {len(simulator.agents)} personas = {len(SCENARIOS) * len(simulator.agents)} total calls")
print()

results = simulator.run_batch(SCENARIOS, structured=True)

print(f"Completed! {len(results)} responses collected.")
results.head(10)

In [ ]:
# Save raw results
results.to_csv(OUTPUT_DIR / 'simulation_results.csv', index=False)
print(f"Saved: {OUTPUT_DIR / 'simulation_results.csv'}")

## 5. Analyze Responses

### 5.1 Decision Summary by Persona

In [ ]:
# Pivot: decisions by persona and scenario
decision_pivot = results.pivot_table(
    index='persona_name',
    columns='scenario_name',
    values='decision',
    aggfunc='first'
)

print("Decision Matrix (Persona × Scenario):")
decision_pivot

In [ ]:
# Count decisions by persona
decision_counts = results.groupby(['persona_name', 'decision']).size().unstack(fill_value=0)
print("Decision counts by persona:")
decision_counts

### 5.2 Response Deep Dive

Examine individual responses to verify persona consistency.

In [ ]:
def display_scenario_responses(scenario_name: str, results_df: pd.DataFrame):
    """Display all persona responses for a given scenario."""
    scenario_results = results_df[results_df['scenario_name'] == scenario_name]
    
    display(Markdown(f"## Scenario: {scenario_name}"))
    display(Markdown("---"))
    
    for _, row in scenario_results.iterrows():
        decision_emoji = "✅" if row['decision'] == 'Yes' else "❌" if row['decision'] == 'No' else "🤔"
        display(Markdown(f"### {row['persona_name']} {decision_emoji}"))
        display(Markdown(f"**Decision**: {row['decision']}"))
        display(Markdown(f"\n{row['response'][:500]}{'...' if len(row['response']) > 500 else ''}"))
        display(Markdown("---"))

In [ ]:
# Examine financing offer responses (tests installment preferences)
display_scenario_responses('financing_offer', results)

In [ ]:
# Examine mixed reviews responses (tests risk tolerance)
display_scenario_responses('mixed_reviews', results)

### 5.3 Visualize Decision Patterns

In [ ]:
# Create decision heatmap
# Map decisions to numeric for visualization
decision_map = {'Yes': 1, 'Maybe': 0.5, 'No': 0, 'Unclear': 0.25}
decision_numeric = decision_pivot.replace(decision_map)

fig, ax = plt.subplots(figsize=(12, 8))
sns.heatmap(
    decision_numeric,
    annot=decision_pivot.values,
    fmt='',
    cmap='RdYlGn',
    center=0.5,
    linewidths=0.5,
    ax=ax,
    cbar_kws={'label': 'Decision (Yes=1, No=0)'}
)
ax.set_title('Persona Decisions Across Scenarios', fontsize=14)
ax.set_xlabel('Scenario', fontsize=12)
ax.set_ylabel('Persona', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'decision_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved: {OUTPUT_DIR / 'decision_heatmap.png'}")

## 6. Validate Persona Differentiation

Check if personas respond consistently with their defining characteristics.

In [ ]:
def validate_persona_consistency(results_df: pd.DataFrame) -> dict:
    """
    Validate that personas respond as expected based on their profiles.
    
    Returns dict with validation results for key persona-scenario pairs.
    """
    validations = []
    
    # Expected behaviors based on persona profiles
    expectations = [
        {
            "persona": "Cash Customer",
            "scenario": "financing_offer",
            "expected": "Should prefer boleto/upfront payment over installments",
            "check_keywords": ["boleto", "upfront", "discount", "cash", "pay now"]
        },
        {
            "persona": "High-Value Financing Shopper",
            "scenario": "financing_offer",
            "expected": "Should prefer installment option, focus on monthly cost",
            "check_keywords": ["installment", "monthly", "spread", "payment plan"]
        },
        {
            "persona": "Bulk Buyer",
            "scenario": "bundle_deal",
            "expected": "Should prefer the bundle over single item",
            "check_keywords": ["bundle", "three", "save", "stock up"]
        },
        {
            "persona": "Critical Shopper",
            "scenario": "mixed_reviews",
            "expected": "Should be hesitant due to negative reviews",
            "check_keywords": ["concern", "risk", "issue", "hesitant", "wait", "no"]
        },
        {
            "persona": "Loyal Explorer Customer",
            "scenario": "new_category",
            "expected": "Should be open to trying new category",
            "check_keywords": ["try", "open", "trust", "explore", "yes"]
        },
        {
            "persona": "Weekend Buyer",
            "scenario": "weekend_promo",
            "expected": "Should prefer waiting for weekend deal",
            "check_keywords": ["wait", "weekend", "discount", "saturday", "sunday"]
        }
    ]
    
    for exp in expectations:
        row = results_df[
            (results_df['persona_name'] == exp['persona']) & 
            (results_df['scenario_name'] == exp['scenario'])
        ]
        
        if row.empty:
            continue
            
        response = row.iloc[0]['response'].lower()
        decision = row.iloc[0]['decision']
        
        # Check if any expected keywords appear
        keywords_found = [kw for kw in exp['check_keywords'] if kw in response]
        
        validations.append({
            "persona": exp['persona'],
            "scenario": exp['scenario'],
            "expectation": exp['expected'],
            "decision": decision,
            "keywords_found": keywords_found,
            "aligned": len(keywords_found) > 0
        })
    
    return validations


# Run validation
validations = validate_persona_consistency(results)
validation_df = pd.DataFrame(validations)

print("Persona Consistency Validation:")
print("=" * 60)
for v in validations:
    status = "✅" if v['aligned'] else "❓"
    print(f"\n{status} {v['persona']} on {v['scenario']}")
    print(f"   Expected: {v['expectation']}")
    print(f"   Decision: {v['decision']}")
    print(f"   Keywords found: {v['keywords_found']}")

In [ ]:
# Summary statistics
if len(validation_df) > 0:
    alignment_rate = validation_df['aligned'].mean() * 100
    print(f"\nOverall alignment rate: {alignment_rate:.0f}%")
    print(f"({validation_df['aligned'].sum()}/{len(validation_df)} expectations met)")

## 7. Summary

In [ ]:
print("\n" + "="*60)
print("SIMULATION COMPLETE")
print("="*60)

print(f"\nSimulation Summary:")
print(f"  - Personas tested: {len(simulator.agents)}")
print(f"  - Scenarios run: {len(SCENARIOS)}")
print(f"  - Total responses: {len(results)}")
print(f"  - Mode: {'Mock' if simulator.mock_mode else 'Live API'}")

print(f"\nOutput files:")
print(f"  - simulation_results.csv")
print(f"  - decision_heatmap.png")

print(f"\nNext Steps:")
print(f"  1. Set ANTHROPIC_API_KEY and change mock_mode=False for real responses")
print(f"  2. Add more scenarios to probe specific persona behaviors")
print(f"  3. Refine persona prompts based on simulation results")